add flags for spikes
clip outlier to 5% to 95% quantile

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display, Markdown
# tripple exponential smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
# mape
from sklearn.metrics import mean_absolute_percentage_error

import optuna


pd.set_option('plotting.backend', 'plotly')

sns.set_theme(style="whitegrid")

pio.renderers.default = "notebook_connected"
# png_renderer = pio.renderers["png"]
# png_renderer.width = 1200
# png_renderer.height = 500

In [2]:
df = pd.read_parquet("../../data/processed/train_enhanced.parquet")
train_location_type_agg = pd.read_parquet("../../data/processed/train_location_type_agg.parquet")
train_region_code_agg = pd.read_parquet("../../data/processed/train_region_code_agg.parquet")
train_store_type_agg = pd.read_parquet("../../data/processed/train_store_type_agg.parquet")
holiday_df= pd.read_csv("../../data/processed/holidays.csv")

In [3]:
def split_train_test(df, test_size=0.2):
    train_size = int(len(df) * (1 - test_size))
    return df.iloc[:train_size], df.iloc[train_size:]

In [4]:
train_location_type_agg

,Date,Location_Type,Total_Sales,Avg_Sales,Total_Orders,Avg_Orders,Num_Stores,Holiday,Total_Discounts
0,2018-01-01,L5,1170560,43354,1444,53,27,1,27
1,2018-01-01,L3,2460245,42418,3175,55,58,1,58
2,2018-01-01,L2,4117915,43808,5295,56,94,1,94
3,2018-01-01,L1,6754876,40939,8662,52,165,1,165
4,2018-01-01,L4,841889,40090,1090,52,21,1,21
...,...,...,...,...,...,...,...,...,...
2575,2019-05-31,L5,642061,23780,1009,37,27,1,1
2576,2019-05-31,L3,1877677,32374,2883,50,58,1,5
2577,2019-05-31,L1,6926767,41980,10286,62,165,1,13
2578,2019-05-31,L2,5582392,59387,8524,91,94,1,20


In [5]:
train_location_type_agg_L1 = train_location_type_agg.loc[train_location_type_agg["Location_Type"] == "L1"]
train_location_type_agg_L1 = train_location_type_agg_L1.set_index("Date")

In [6]:
train_location_type_agg_L1_train, train_location_type_agg_L1_test = split_train_test(train_location_type_agg_L1, test_size=0.2)

In [7]:
len(train_location_type_agg_L1_test)

104

In [8]:
try:
    ses = ExponentialSmoothing(train_location_type_agg_L1_train["Total_Sales"], seasonal="additive", seasonal_periods=150, freq='D').fit()
    ses_pred = ses.forecast(len(train_location_type_agg_L1_test))
except Exception as e:
    print("Optimization failed to converge:", e)
    print("MLE return values:", ses.mle_retvals)

fig = go.Figure()
fig.add_trace(go.Scatter(x=train_location_type_agg_L1_train.index, y=train_location_type_agg_L1_train["Total_Sales"], mode='lines', name='Train'))
fig.add_trace(go.Scatter(x=train_location_type_agg_L1_test.index, y=train_location_type_agg_L1_test["Total_Sales"], mode='lines', name='Test'))
fig.add_trace(go.Scatter(x=train_location_type_agg_L1_test.index, y=ses_pred, mode='lines', name='Forecast'))
fig.show()

/opt/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



In [9]:
ses.mle_retvals

 message: Inequality constraints incompatible
 success: False
  status: 4
     fun: 903066914929497.8
       x: [ 7.571e-02  2.029e-01 ...  1.135e+06  2.633e+05]
     nit: 1
     jac: [-4.636e+13 -8.307e+11 ...  0.000e+00  0.000e+00]
    nfev: 154
    njev: 1

In [10]:
mean_absolute_percentage_error(train_location_type_agg_L1_test["Total_Sales"], ses_pred)

0.2230375152128284